In [9]:
from rdflib import Dataset, URIRef
from rdflib import Namespace
import csv

In [10]:
datapath = "/home/jupyter-mvogl/mydata/"

In [11]:
CRM = Namespace('http://cidoc-crm.org/cidoc-crm/')

In [12]:
# the URI for our named graph that will contain our data
anno_named_graph_uri = 'http://example.org/ns/users/malte/genesis-example-geo2'
anno_named_graph_uriref = URIRef(anno_named_graph_uri)

In [13]:
# open a connection to the Blazegraph triple store
blazegraph_uri = 'http://localhost:8080/blazegraph/namespace/kb/sparql'
store = Dataset('SPARQLUpdateStore')
store.open((blazegraph_uri, blazegraph_uri))

In [14]:
# find our named graph if it exists or create a new one
anno_named_graph = None
for g in store.graphs():
    if g.identifier == anno_named_graph_uriref:
        anno_named_graph = g

if anno_named_graph is None:
    anno_named_graph = store.graph(anno_named_graph_uriref)

In [15]:
# read relations from CSV file
with open(f'{datapath}gtclbuppjmot3c_relations(1).csv', newline='') as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        source_name = row['source']
        target_name = row['target']
        rel_type = row['relation']
        print(f"relation: {source_name} - {rel_type} -> {target_name}")
        
        if rel_type == 'father_of':
            source_person = URIRef(f"{anno_named_graph_uri}/persons/{source_name}")
            target_person = URIRef(f"{anno_named_graph_uri}/persons/{target_name}")
            # create father_of as crm:P152_has_parent relation
            anno_named_graph.add((target_person, CRM.P152_has_parent, source_person))
        elif rel_type == 'river_in':
            source_person = URIRef(f"{anno_named_graph_uri}/rivers/{source_name}")
            target_person = URIRef(f"{anno_named_graph_uri}/places/{target_name}")
            # create father_of as crm:P152_has_parent relation
            anno_named_graph.add((target_person, CRM.P53_has_location, source_person))
            
            # we could add a relation linked to the concrete annotations in the source
            #source_id = row['source_id']
            #target_id = row['target_id']
            #target_anno = URIRef(f"https://recogito.pelagios.org/annotation/{target_id}")
            #source_anno = URIRef(f"https://recogito.pelagios.org/annotation/{source_id}")
            
        else:
            print(f"don't know what to do with {rel_type=}")
            pass

relation: Gihon - river_in -> Cush
relation: Mehujael - father_of -> Methushael
relation: Enoch - father_of -> Irad
relation: Lamech - father_of -> Noah
relation: Ham - father_of -> Cush
relation: Ham - father_of -> Mizraim
relation: Ham - father_of -> Canaan
relation: Shem - father -> Elam
don't know what to do with rel_type='father'
relation: Shem - father_of -> Asshur
relation: Shem - father_of -> Arpachshad
relation: Shem - father_of -> Aram
relation: Jared - father_of -> Enoch
relation: Hiddekel - river_in -> Assyria
relation: Cush - father_of -> Nimrod
relation: Pishon - river_in -> Havilah
relation: Methushael - father_of -> Lamech
relation: Seth - father_of -> Enosh
relation: Canaanites - lived_in -> Sidon
don't know what to do with rel_type='lived_in'
relation: Canaanites - lived_in -> Gerar
don't know what to do with rel_type='lived_in'
relation: Canaanites - lived_in -> Gaza
don't know what to do with rel_type='lived_in'
relation: Japheth - father_of -> Gomer
relation: Japhe

In [16]:
store.close()